In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [10]:
# initialize list of lists 
cities = [['Dunwoody', 'GA'], ['Sandy Springs', 'GA'], ['Roswell','GA'],['Woodstock','GA'],['Norcross','GA'],['Suwanee','GA'],['Alpharetta','GA'],['Marietta','GA'],['Duluth','GA']] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(cities, columns = ['City', 'State']) 
  
# print dataframe. 
df 

,City,State
0,Dunwoody,GA
1,Sandy Springs,GA
2,Roswell,GA
3,Woodstock,GA
4,Norcross,GA
5,Suwanee,GA
6,Alpharetta,GA
7,Marietta,GA
8,Duluth,GA


In [11]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, GA'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [12]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["City"].tolist() ]

In [13]:
coords

[[33.92620000000005, -84.33617999999996],
 [33.92483000000004, -84.37907999999999],
 [34.021660000000054, -84.35929999999996],
 [34.102910000000065, -84.51874999999995],
 [33.942810000000065, -84.21094999999997],
 [34.05746000000005, -84.06474999999995],
 [34.07540000000006, -84.29454999999996],
 [33.95319000000006, -84.54587999999995],
 [34.003630000000044, -84.14536999999996]]

In [14]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [15]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [16]:
# check the neighborhoods and the coordinates
print(df.shape)
df

(9, 4)


,City,State,Latitude,Longitude
0,Dunwoody,GA,33.92620,-84.33618
1,Sandy Springs,GA,33.92483,-84.37908
2,Roswell,GA,34.02166,-84.35930
3,Woodstock,GA,34.10291,-84.51875
4,Norcross,GA,33.94281,-84.21095
5,Suwanee,GA,34.05746,-84.06475
6,Alpharetta,GA,34.07540,-84.29455
7,Marietta,GA,33.95319,-84.54588
8,Duluth,GA,34.00363,-84.14537


In [17]:
# get the coordinates of Roswell, GA
address = 'Roswell, GA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Roswell, GA {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Roswell, GA 34.02332, -84.360022.


In [18]:
# create map of Atlanta Cities using Roswell latitude and longitude values as the center
map_atl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_atl)  
    
map_atl

In [19]:
# define Foursquare Credentials and Version
CLIENT_ID = 'MKIX5GO0RCMABWJ2HYMBTLBHAMHSPZLQTVSXOEH24SA5BHG4' # your Foursquare ID
CLIENT_SECRET = 'GNFV4N01EC4LK2EJEMVD25J2UNKCR3COGSBNZ4LWO30JFLVX' # your Foursquare Secret
VERSION = '20180602' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MKIX5GO0RCMABWJ2HYMBTLBHAMHSPZLQTVSXOEH24SA5BHG4
CLIENT_SECRET:GNFV4N01EC4LK2EJEMVD25J2UNKCR3COGSBNZ4LWO30JFLVX


In [59]:
radius = 2000
LIMIT = 100
CATEGORYID = '4d4b7105d754a06374d81259'

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['City']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4d4b7105d754a06374d81259".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT,
        CATEGORYID
    )
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [60]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['City', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(50)

(525, 7)


,City,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Dunwoody,33.9262,-84.33618,Alon's Bakery & Market,33.924560,-84.336402,Bakery
1,Dunwoody,33.9262,-84.33618,The Capital Grille,33.926564,-84.340573,American Restaurant
2,Dunwoody,33.9262,-84.33618,Seasons 52,33.926519,-84.339795,New American Restaurant
3,Dunwoody,33.9262,-84.33618,McKendrick's Steak House,33.925110,-84.335980,Steakhouse
4,Dunwoody,33.9262,-84.33618,Eclipse di Luna,33.924995,-84.335298,Tapas Restaurant
5,Dunwoody,33.9262,-84.33618,Fogo de Chão,33.930973,-84.337048,Steakhouse
6,Dunwoody,33.9262,-84.33618,Corner Bakery Cafe,33.927961,-84.336716,Bakery
7,Dunwoody,33.9262,-84.33618,California Pizza Kitchen at Perimeter,33.928364,-84.337520,Pizza Place
8,Dunwoody,33.9262,-84.33618,Maggiano's Little Italy,33.923520,-84.340550,Italian Restaurant
9,Dunwoody,33.9262,-84.33618,Cafe Intermezzo,33.925109,-84.335703,Café


In [92]:
venues_subcount= venues_df.groupby(["City","VenueCategory"]).count()
venues_subcount.head(40)


Latitude  Longitude  VenueName  \
City       VenueCategory                                               
Alpharetta American Restaurant               6          6          6   
           BBQ Joint                         1          1          1   
           Bagel Shop                        3          3          3   
           Breakfast Spot                    2          2          2   
           Burger Joint                      2          2          2   
           Café                              2          2          2   
           Cajun / Creole Restaurant         1          1          1   
           Deli / Bodega                     3          3          3   
           Donut Shop                        1          1          1   
           Falafel Restaurant                1          1          1   
           Fast Food Restaurant              4          4          4   
           Food                              1          1          1   
           Food Truck                        1          1          1   
           French Restaurant                 1          1          1   
           Fried Chicken Joint               1          1          1   
           Gastropub                         1          1          1   
           German Restaurant                 1          1          1   
           Greek Restaurant                  1          1          1   
           Indian Restaurant                 3          3          3   
           Italian Restaurant                3          3          3   
           Japanese Restaurant               3          3          3   
           Mediterranean Restaurant          1          1          1   
           Mexican Restaurant                4          4          4   
           Middle Eastern Restaurant         1          1          1   
           New American Restaurant           6          6          6   
           North Indian Restaurant           1          1          1   
           Pizza Place                       6          6          6   
           Poke Place                        1          1          1   
           Restaurant                        3          3          3   
           Salad Place                       1          1          1   
           Sandwich Place                    2          2          2   
           Seafood Restaurant                1          1          1   
           Steakhouse                        2          2          2   
           Sushi Restaurant                  1          1          1   
           Taco Place                        3          3          3   
           Tapas Restaurant                  1          1          1   
           Tex-Mex Restaurant                1          1          1   
           Thai Restaurant                   1          1          1   
           Vietnamese Restaurant             1          1          1   
Duluth     American Restaurant               1          1          1   

                                      VenueLatitude  VenueLongitude  
City       VenueCategory                                             
Alpharetta American Restaurant                    6               6  
           BBQ Joint                              1               1  
           Bagel Shop                             3               3  
           Breakfast Spot                         2               2  
           Burger Joint                           2               2  
           Café                                   2               2  
           Cajun / Creole Restaurant              1               1  
           Deli / Bodega                          3               3  
           Donut Shop                             1               1  
           Falafel Restaurant                     1               1  
           Fast Food Restaurant                   4               4  
           Food                                   1               1  
           Food Truck            

In [99]:
venues_sum= venues_subcount.groupby(['City']).count()
venues_sum.head(20)


Latitude  Longitude  VenueName  \
City       VenueCategory                                               
Alpharetta American Restaurant               1          1          1   
           BBQ Joint                         1          1          1   
           Bagel Shop                        1          1          1   
           Breakfast Spot                    1          1          1   
           Burger Joint                      1          1          1   
           Café                              1          1          1   
           Cajun / Creole Restaurant         1          1          1   
           Deli / Bodega                     1          1          1   
           Donut Shop                        1          1          1   
           Falafel Restaurant                1          1          1   
           Fast Food Restaurant              1          1          1   
           Food                              1          1          1   
           Food Truck                        1          1          1   
           French Restaurant                 1          1          1   
           Fried Chicken Joint               1          1          1   
           Gastropub                         1          1          1   
           German Restaurant                 1          1          1   
           Greek Restaurant                  1          1          1   
           Indian Restaurant                 1          1          1   
           Italian Restaurant                1          1          1   

                                      VenueLatitude  VenueLongitude  
City       VenueCategory                                             
Alpharetta American Restaurant                    1               1  
           BBQ Joint                              1               1  
           Bagel Shop                             1               1  
           Breakfast Spot                         1               1  
           Burger Joint                           1               1  
           Café                                   1               1  
           Cajun / Creole Restaurant              1               1  
           Deli / Bodega                          1               1  
           Donut Shop                             1               1  
           Falafel Restaurant                     1               1  
           Fast Food Restaurant                   1               1  
           Food                                   1               1  
           Food Truck                             1               1  
           French Restaurant                      1               1  
           Fried Chicken Joint                    1               1  
           Gastropub                              1               1  
           German Restaurant                      1               1  
           Greek Restaurant                       1               1  
           Indian Restaurant                      1               1  
           Italian Restaurant                     1               1

In [90]:
pd.merge(venues_subcount, venues_sum, on='City', how='outer')

TypeError: Can only merge Series or DataFrame objects, a <class 'pandas.core.groupby.generic.DataFrameGroupBy'> was passed

In [68]:
# install wordcloud
!conda install -c conda-forge wordcloud==1.4.1 --yes

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - wordcloud==1.4.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wordcloud-1.4.1            |           py36_0         324 KB  conda-forge

The following NEW packages will be INSTALLED:

    wordcloud: 1.4.1-py36_0 conda-forge


wordcloud-1.4.1      | 324 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Wordcloud is installed and imported!


In [ ]:
# display the cloud
fig = plt.figure()
fig.set_figwidth(14)
fig.set_figheight(18)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [67]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 67 uniques categories.


In [27]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:75]

array(['Bakery', 'American Restaurant', 'New American Restaurant',
       'Steakhouse', 'Tapas Restaurant', 'Italian Restaurant',
       'Pizza Place', 'Café', 'Sandwich Place', 'Deli / Bodega',
       'Breakfast Spot', 'Mexican Restaurant', 'Donut Shop',
       'Sushi Restaurant', 'Fast Food Restaurant', 'Taco Place',
       'Burger Joint', 'French Restaurant', 'Mediterranean Restaurant',
       'Indian Restaurant', 'Asian Restaurant', 'Hawaiian Restaurant',
       'BBQ Joint', 'Chinese Restaurant', 'Burrito Place', 'Food Truck',
       'Buffet', 'Food Court', 'Bagel Shop', 'Middle Eastern Restaurant',
       'Wings Joint', 'Korean Restaurant', 'Greek Restaurant',
       'Thai Restaurant', 'Tex-Mex Restaurant', 'Vietnamese Restaurant',
       'Restaurant', 'Cafeteria', 'Ramen Restaurant', 'Noodle House',
       'Vegetarian / Vegan Restaurant', 'Latin American Restaurant',
       'Hot Dog Joint', 'Caribbean Restaurant', 'Japanese Restaurant',
       'Fried Chicken Joint', 'Food', 'Sala

In [28]:
# one hot encoding
atl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add City column back to dataframe
atl_onehot['Cities'] = venues_df['City'] 

# move City column to the first column
fixed_columns = [atl_onehot.columns[-1]] + list(atl_onehot.columns[:-1])
atl_onehot = atl_onehot[fixed_columns]

print(atl_onehot.shape)
atl_onehot.head(20)

(861, 72)


,Cities,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Eastern European Restaurant,Empanada Restaurant,Fast Food Restaurant,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mac & Cheese Joint,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Snack Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Dunwoody,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dunwoody,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dunwoody,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dunwoody,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Dunwoody,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,Dunwoody,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,Dunwoody,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Dunwoody,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Dunwoody,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Dunwoody,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
atl_grouped = atl_onehot.groupby(["Cities"]).mean().reset_index()

print(atl_grouped.shape)
atl_grouped

(9, 72)


,Cities,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Eastern European Restaurant,Empanada Restaurant,Fast Food Restaurant,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mac & Cheese Joint,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Snack Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Alpharetta,0.070000,0.00,0.020000,0.010000,0.02,0.020000,0.00,0.050000,0.01,0.040000,0.00,0.000000,0.030000,0.010000,0.000000,0.000000,0.010000,0.00,0.01,0.040000,0.000000,0.020000,0.000000,0.00,0.070000,0.000000,0.020000,0.000000,0.010000,0.01,0.010000,0.010000,0.000000,0.020000,0.000000,0.00,0.000000,0.010000,0.000000,0.040000,0.020000,0.010000,0.00,0.000000,0.030000,0.070000,0.01,0.070000,0.00,0.01,0.030000,0.000000,0.00,0.020000,0.00,0.020000,0.000000,0.010000,0.00,0.00,0.010000,0.020000,0.010000,0.030000,0.01,0.010000,0.030000,0.00,0.000000,0.010000,0.010000
1,Duluth,0.031915,0.00,0.031915,0.010638,0.00,0.021277,0.00,0.042553,0.00,0.000000,0.00,0.010638,0.031915,0.010638,0.000000,0.010638,0.095745,0.00,0.00,0.000000,0.010638,0.031915,0.000000,0.00,0.053191,0.010638,0.000000,0.000000,0.021277,0.00,0.010638,0.000000,0.010638,0.010638,0.010638,0.00,0.010638,0.031915,0.000000,0.010638,0.010638,0.063830,0.00,0.010638,0.010638,0.053191,0.00,0.000000,0.00,0.00,0.095745,0.000000,0.00,0.063830,0.00,0.053191,0.010638,0.000000,0.00,0.00,0.021277,0.010638,0.031915,0.010638,0.00,0.000000,0.021277,0.00,0.000000,0.010638,0.000000
2,Dunwoody,0.100000,0.00,0.020000,0.020000,0.01,0.050000,0.00,0.050000,0.01,0.030000,0.02,0.010000,0.030000,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.060000,0.000000,0.020000,0.000000,0.00,0.020000,0.000000,0.010000,0.000000,0.020000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000,0.030000,0.000000,0.050000,0.000000,0.010000,0.00,0.000000,0.020000,0.040000,0.01,0.020000,0.00,0.00,0.080000,0.000000,0.00,0.010000,0.00,0.040000,0.000000,0.000000,0.00,0.00,0.000000,0.040000,0.020000,0.010000,0.02,0.010000,0.020000,0.00,0.000000,0.010000,0.030000
3,Marietta,0.060000,0.00,0.010000,0.050000,0.00,0.060000,0.00,0.030000,0.00,0.020000,0.01,0.000000,0.030000,0.000000,0.010000,0.000000,0.020000,0.00,0.00,0.040000,0.010000,0.030000,0.000000,0.00,0.070000,0.020000,0.010000,0.000000,0.000000,0.03,0.050000,0.000000,0.000000,0.010000,0.000000,0.00,0.020000,0.000000,0.010000,0.010000,0.000000,0.010000,0.02,0.000000,0.000000,0.050000,0.00,0.000000,0.00,0.00,0.050000,0.000000,0.00,0.040000,0.00,0.110000,0.020000,0.000000,0.01,0.00,0.030000,0.000000,0.000000,0.010000,0.00,0.000000,0.010000,0.00,0.000000,0.000000,0.030000
4,Norcross,0.070000,0.01,0.020000,0.020000,0.00,0.030000,0.01,0.060000,0.00,0.010000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.030000,0.02,0.01,0.010000,0.000000,0.030000,0.000000,0.01,0.080000,0.010000,0.000000,0.000000,0.000000,0.00,0.020000,0.010000,0.000000,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.010000,0.030000,0.000000,0.01,0.000000,0.020000,0.100000,0.00,0.000000,0.00,0.00,0.060000,0.010000,0.00,0.040000,0.00,0.080000,0.020000,0.000000,0.01,0.01,0.000000,0.010000,0.000000,0.010000,0.00,0.000

In [20]:
atl_thai = atl_grouped[["Cities","Thai Restaurant"]]

In [21]:
atl_thai.head()

,Cities,Thai Restaurant
0,Alpharetta,0.030000
1,Duluth,0.022222
2,Dunwoody,0.030000
3,Marietta,0.010000
4,Norcross,0.020202


In [22]:
# set number of clusters
kclusters = 3

atl_clustering = atl_thai.drop(["Cities"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(atl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 1, 2, 0, 1, 2, 1], dtype=int32)

In [23]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
atl_merged = atl_thai.copy()

# add clustering labels
atl_merged["Cluster Labels"] = kmeans.labels_



In [24]:
atl_merged.rename(columns={"Cities": "City"}, inplace=True)
atl_merged.head()

,City,Thai Restaurant,Cluster Labels
0,Alpharetta,0.030000,0
1,Duluth,0.022222,2
2,Dunwoody,0.030000,0
3,Marietta,0.010000,1
4,Norcross,0.020202,2


In [25]:
# merge atlanta_grouped with atlanta_data to add latitude/longitude for each neighborhood
atl_merged = atl_merged.join(df.set_index("City"), on="City")

print(atl_merged.shape)
atl_merged.head() # check the last columns!

(10, 6)


,City,Thai Restaurant,Cluster Labels,State,Latitude,Longitude
0,Alpharetta,0.030000,0,GA,34.07540,-84.29455
1,Duluth,0.022222,2,GA,34.00363,-84.14537
2,Dunwoody,0.030000,0,GA,33.92620,-84.33618
3,Marietta,0.010000,1,GA,33.95319,-84.54588
3,Marietta,0.010000,1,GA,33.95319,-84.54588


In [26]:
# sort the results by Cluster Labels
print(atl_merged.shape)
atl_merged.sort_values(["Cluster Labels"], inplace=True)
atl_merged



(10, 6)


,City,Thai Restaurant,Cluster Labels,State,Latitude,Longitude
0,Alpharetta,0.030000,0,GA,34.07540,-84.29455
2,Dunwoody,0.030000,0,GA,33.92620,-84.33618
5,Roswell,0.030000,0,GA,34.02166,-84.35930
3,Marietta,0.010000,1,GA,33.95319,-84.54588
3,Marietta,0.010000,1,GA,33.95319,-84.54588
6,Sandy Springs,0.010000,1,GA,33.92483,-84.37908
8,Woodstock,0.000000,1,GA,34.10291,-84.51875
1,Duluth,0.022222,2,GA,34.00363,-84.14537
4,Norcross,0.020202,2,GA,33.94281,-84.21095
7,Suwanee,0.024096,2,GA,34.05746,-84.06475


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(atl_merged['Latitude'], atl_merged['Longitude'], atl_merged['City'], atl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
atl_merged.loc[atl_merged['Cluster Labels'] == 0]

,City,Thai Restaurant,Cluster Labels,State,Latitude,Longitude
0,Alpharetta,0.03,0,GA,34.07540,-84.29455
2,Dunwoody,0.03,0,GA,33.92620,-84.33618
5,Roswell,0.03,0,GA,34.02166,-84.35930


In [29]:
atl_merged.loc[atl_merged['Cluster Labels'] == 1]

,City,Thai Restaurant,Cluster Labels,State,Latitude,Longitude
3,Marietta,0.01,1,GA,33.95319,-84.54588
3,Marietta,0.01,1,GA,33.95319,-84.54588
6,Sandy Springs,0.01,1,GA,33.92483,-84.37908
8,Woodstock,0.00,1,GA,34.10291,-84.51875


In [31]:
atl_merged.loc[atl_merged['Cluster Labels'] == 2]

,City,Thai Restaurant,Cluster Labels,State,Latitude,Longitude
1,Duluth,0.022222,2,GA,34.00363,-84.14537
4,Norcross,0.020202,2,GA,33.94281,-84.21095
7,Suwanee,0.024096,2,GA,34.05746,-84.06475


In [32]:
# install wordcloud
!conda install -c conda-forge wordcloud==1.4.1 --yes

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - wordcloud==1.4.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wordcloud-1.4.1            |           py36_0         324 KB  conda-forge

The following NEW packages will be INSTALLED:

    wordcloud: 1.4.1-py36_0 conda-forge


wordcloud-1.4.1      | 324 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Wordcloud is installed and imported!


In [33]:
venues_df.head()

,City,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Dunwoody,33.9262,-84.33618,Alon's Bakery & Market,33.924560,-84.336402,Bakery
1,Dunwoody,33.9262,-84.33618,The Capital Grille,33.926564,-84.340573,American Restaurant
2,Dunwoody,33.9262,-84.33618,Seasons 52,33.926519,-84.339795,New American Restaurant
3,Dunwoody,33.9262,-84.33618,McKendrick's Steak House,33.925110,-84.335980,Steakhouse
4,Dunwoody,33.9262,-84.33618,Eclipse di Luna,33.924995,-84.335298,Tapas Restaurant


In [ ]:
total_immigration = df_can['Total'].sum()
total_immigration

In [35]:
max_words = 90
word_string = ''
for country in df_can.index.values:
    # check if country's name is a single-word name
    if len(country.split(' ')) == 1:
        repeat_num_times = int(df_can.loc[country, 'Total']/float(total_immigration)*max_words)
        word_string = word_string + ((country + ' ') * repeat_num_times)
                                     
# display the generated text
word_string

NameError: name 'df_can' is not defined

In [36]:
# create the word cloud
wordcloud = WordCloud(background_color='white').generate(word_string)

print('Word cloud created!')

ValueError: We need at least 1 word to plot a word cloud, got 0.

In [37]:
# display the cloud
fig = plt.figure()
fig.set_figwidth(14)
fig.set_figheight(18)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

NameError: name 'plt' is not defined